In [ ]:
"""
Real-time HMI: Air Canvas (Gesture Drawing Board)

Features:

1. OpenCV captures camera frame at 640x480 resolution. The camera is initialized to robustly handle macOS's Continuity Camera by checking multiple indices.
2. PyQt label displays the live camera feed combined with the persistent Canvas Layer.
3. MediaPipe detects 21 hand landmarks in VIDEO mode (synchronous).
4. HMI Gesture Logic: The system detects a "Pinch Gesture" (Index Finger #8 close to Thumb #4) to switch the system state between DRAWING (active) and MOVING (inactive).
5. Drawing Layer: Strokes are drawn on a separate, persistent NumPy array (Canvas Layer), which is then blended with the live video frame.
6. FPS is computed as average frames per second (stable measurement).
7. Motion is smoothed using Exponential Smoothing (α=0.3) to reduce jitter in the drawn lines.

Author: Dimas Ahmad Fahreza 113021199
Course: Human–Machine Interface
Date: October 2025

"""

import sys
import time
import cv2
import numpy as np
from PyQt6.QtWidgets import QApplication, QLabel, QWidget, QVBoxLayout, QPushButton, QHBoxLayout
from PyQt6.QtGui import QImage, QPixmap
from PyQt6.QtCore import Qt, QTimer

import mediapipe as mp
from mediapipe.tasks import python as mp_python
from mediapipe.tasks.python import vision as mp_vision

# ============================================================
# I. Initialize MediaPipe HandLandmarker in VIDEO mode
# ============================================================

# Pastikan file 'hand_landmarker.task' ada di direktori yang sama
MODEL_PATH = "hand_landmarker.task"

try:
    options = mp_vision.HandLandmarkerOptions(
        base_options=mp_python.BaseOptions(model_asset_path=MODEL_PATH),
        running_mode=mp_vision.RunningMode.VIDEO,
        num_hands=1
    )
    hand_landmarker = mp_vision.HandLandmarker.create_from_options(options)
except FileNotFoundError:
    print(f"ERROR: Model file '{MODEL_PATH}' not found. Please ensure it is in the same directory.")
    sys.exit(1)
except Exception as e:
    print(f"ERROR initializing MediaPipe: {e}")
    sys.exit(1)

# ============================================================
# II. PyQt6 GUI Application (Air Canvas)
# ============================================================

class AirCanvasApp(QWidget):
    def __init__(self):
        super().__init__()
        self.setWindowTitle("HMI Air Canvas - Gesture Drawing")
        
        # Fixed Resolution for consistency
        self.FRAME_W, self.FRAME_H = 640, 480
        self.setGeometry(100, 100, self.FRAME_W, self.FRAME_H + 50) 
        
        # --- UI Components ---
        self.video_label = QLabel(self)
        self.video_label.setAlignment(Qt.AlignmentFlag.AlignCenter)

        self.clear_button = QPushButton("Clear Canvas")
        self.clear_button.clicked.connect(self.clear_canvas)
        
        control_layout = QHBoxLayout()
        control_layout.addWidget(self.clear_button)
        
        main_layout = QVBoxLayout()
        main_layout.addWidget(self.video_label)
        main_layout.addLayout(control_layout)
        self.setLayout(main_layout)

        self.cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
        
        if not self.cap.isOpened():
            self.cap = cv2.VideoCapture(0, cv2.CAP_AVFOUNDATION)
            if not self.cap.isOpened():
                raise RuntimeError("Cannot open webcam. Check system permissions or available camera index.")
        
        self.cap.set(cv2.CAP_PROP_FRAME_WIDTH, self.FRAME_W)
        self.cap.set(cv2.CAP_PROP_FRAME_HEIGHT, self.FRAME_H)
            
        # --- Canvas and Drawing Variables ---
        self.canvas_layer = np.zeros((self.FRAME_H, self.FRAME_W, 3), dtype=np.uint8)
        
        self.current_x, self.current_y = 0, 0
        self.prev_x, self.prev_y = 0, 0
        self.is_drawing = False
        
        self.SMOOTHING_FACTOR = 0.3
        self.PINCH_THRESHOLD = 50 

        # --- Performance Variables ---
        self.fps_start_time = time.time()
        self.frame_count = 0
        self.fps = 0.0
        self.frame_timestamp = 0
        
        # --- QTimer Loop ---
        self.timer = QTimer()
        self.timer.timeout.connect(self.update_frame)
        self.timer.start(33)

    # -----------------------------------------------
    # Utility Function: Clear the Drawing Layer
    # -----------------------------------------------
    def clear_canvas(self):
        """Reset the canvas layer to a blank black image."""
        self.canvas_layer = np.zeros((self.FRAME_H, self.FRAME_W, 3), dtype=np.uint8)
        print("Canvas cleared.")

    # -----------------------------------------------
    # Main Loop: Capture, Process, and Display
    # -----------------------------------------------
    def update_frame(self):
        # 1. Capture one frame from webcam
        ret, frame = self.cap.read()
        if not ret:
            # Re-initialize capture if it fails (common on Mac sleep/wake)
            self.cap = cv2.VideoCapture(1, cv2.CAP_AVFOUNDATION)
            if not self.cap.isOpened():
                return
            ret, frame = self.cap.read()
            if not ret: return

        # 2. Preprocessing
        frame = cv2.flip(frame, 1) # Mirror effect
        h, w, _ = frame.shape
        
        rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        mp_image = mp.Image(image_format=mp.ImageFormat.SRGB, data=rgb_frame)
        
        # 3. Update MediaPipe timestamp (VIDEO mode requirement)
        self.frame_timestamp += 33 

        # 4. Run detector
        results = hand_landmarker.detect_for_video(mp_image, self.frame_timestamp)

        # 5. Drawing and Smoothing Logic
        if results.hand_landmarks:
            for hand_landmarks in results.hand_landmarks:
                # Landmark 8: Index Fingertip; Landmark 4: Thumb Tip
                x8 = int(hand_landmarks[8].x * w)
                y8 = int(hand_landmarks[8].y * h)
                x4 = int(hand_landmarks[4].x * w)
                y4 = int(hand_landmarks[4].y * h)
                
                # a. Apply Exponential Smoothing (for drawing point)
                self.current_x = int((1 - self.SMOOTHING_FACTOR) * self.current_x + self.SMOOTHING_FACTOR * x8)
                self.current_y = int((1 - self.SMOOTHING_FACTOR) * self.current_y + self.SMOOTHING_FACTOR * y8)
                
                # b. Calculate distance for Pinch Detection
                pinch_distance = np.sqrt((x8 - x4)**2 + (y8 - y4)**2)
                
                # c. Determine Drawing Mode
                if pinch_distance < self.PINCH_THRESHOLD:
                    self.is_drawing = True
                    # Draw a visual indicator (red circle) for drawing mode
                    cv2.circle(frame, (self.current_x, self.current_y), 5, (0, 0, 255), -1) 
                else:
                    self.is_drawing = False
                    # Draw a visual indicator (green circle) for moving mode
                    cv2.circle(frame, (self.current_x, self.current_y), 5, (0, 255, 0), -1) 
                    
                # d. Perform Drawing on the Canvas Layer
                if self.is_drawing and self.prev_x != 0:
                    # Draw line on the canvas layer (Blue color, thickness 5)
                    cv2.line(self.canvas_layer, 
                             (self.prev_x, self.prev_y), 
                             (self.current_x, self.current_y), 
                             (255, 0, 0), 
                             thickness=5)
                
                # e. Update previous position for the next frame
                self.prev_x, self.prev_y = self.current_x, self.current_y
                
                # f. Draw ALL landmarks (optional visual feedback)
                for lm in hand_landmarks:
                    cx, cy = int(lm.x * w), int(lm.y * h)
                    cv2.circle(frame, (cx, cy), 2, (0, 255, 255), -1)

        else:
            # If no hand is detected, ensure drawing is off and reset previous position
            self.is_drawing = False
            self.prev_x, self.prev_y = 0, 0


        # 6. Combine the Video Frame and the Canvas Layer
        frame = cv2.addWeighted(frame, 1.0, self.canvas_layer, 1.0, 0)

        # 7. Update and Draw FPS
        self.frame_count += 1
        elapsed = time.time() - self.fps_start_time
        if elapsed > 1.0:
            self.fps = self.frame_count / elapsed
            self.fps_start_time = time.time()
            self.frame_count = 0

        cv2.putText(frame, f"FPS: {self.fps:.1f}", (10, 40),
                    cv2.FONT_HERSHEY_SIMPLEX, 1.0, (255, 255, 0), 2)
        cv2.putText(frame, f"Mode: {'DRAWING' if self.is_drawing else 'MOVING'}", (10, 80),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 255), 2)

        # 8. Convert the final frame to QImage and display
        rgb_display = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        h, w, ch = rgb_display.shape
        bytes_per_line = ch * w
        qimg = QImage(rgb_display.data, w, h, bytes_per_line, QImage.Format.Format_RGB888)
        pixmap = QPixmap.fromImage(qimg)
        self.video_label.setPixmap(pixmap)

    # -----------------------------------------------
    # Cleanup resources
    # -----------------------------------------------
    def closeEvent(self, event):
        self.timer.stop()
        self.cap.release()
        try:
            hand_landmarker.close()
        except Exception:
            pass
        event.accept()

# ============================================================
# III. Main entry point
# ============================================================
if __name__ == "__main__":
    app = QApplication(sys.argv)
    window = AirCanvasApp()
    window.show()
    sys.exit(app.exec())


I0000 00:00:1761807424.953492 15020248 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 89.4), renderer: Apple M1
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
W0000 00:00:1761807425.038415 15033091 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1761807425.081508 15033094 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
2025-10-30 14:57:06.397 Python[46879:15020248] WARNING: AVCaptureDeviceTypeExternal is deprecated for Continuity Cameras. Please use AVCaptureDeviceTypeContinuityCamera and add NSCameraUseContinuityCameraDeviceType to your Info.plist.
W0000 00:00:1761807428.165014 15033092 landmark_projection_calculator.cc:186] Using NORM_RECT without IMAGE_DIMENSIONS is only supported for the square ROI. Provide IMAGE_DIMENSIONS or use PROJECTION_MATRIX.


Canvas cleared.
Canvas cleared.
Canvas cleared.
Canvas cleared.


KeyboardInterrupt: 